In [1]:
# Imports
import torch
from torch.distributions import Normal,Laplace,StudentT
from dask_jobqueue import SLURMCluster
from distributed import Client
import e401k_Cocycles

In [2]:
# Args set up
seeds = torch.linspace(0,99,100).int()
splits = 5
estimators = ["S-estimator"]
cocycles = ["linear","additive","affine","TMI"]
weight_decays = [1e-3]
widths = [32,64,128]
bootstrap = False

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="24GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=300)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in seeds:
    for estimator in estimators:
        for cocycle in cocycles:
            for weight_decay in weight_decays:
                for width in widths:
                    f = client.submit(e401k_Cocycles.run_experiment,seed,estimator,splits,bootstrap,cocycle,width,weight_decay)
                    futures += [f]

In [5]:
futures

[<Future: pending, key: run_experiment-1526b2018b756ed3df9526b50608a418>,
 <Future: pending, key: run_experiment-6eb551a76e2d144234cd8f93de109f85>,
 <Future: pending, key: run_experiment-455d96ef5e9e0b2008e8b47c01a5a23b>,
 <Future: pending, key: run_experiment-c52a9676d35026563e6695728eb5c2c8>,
 <Future: pending, key: run_experiment-06422c00537fa4a7c7320be91c324f80>,
 <Future: pending, key: run_experiment-933390836489756de09fe64d6e4fc836>,
 <Future: pending, key: run_experiment-cd8f2cf21747ca922d6c85c3471994ab>,
 <Future: pending, key: run_experiment-e1d26d50dbf578ebed80f303b9c8844b>,
 <Future: pending, key: run_experiment-54034438f109d9a6e3f9519fa592852b>,
 <Future: pending, key: run_experiment-c37d5e1ee56d6b54e318f6a6ae062c2e>,
 <Future: pending, key: run_experiment-4ff12c6b4a5c430736d5c0004a22b156>,
 <Future: pending, key: run_experiment-3d8a56ce0913765b29275ecdd4df8715>,
 <Future: pending, key: run_experiment-706bbbe048aedebc75f35662a2a3366a>,
 <Future: pending, key: run_experiment

In [6]:
# Getting results
results = client.gather(futures)

In [5]:
# Closing client
client.close()
cluster.close()

2024-09-16 11:16:57,999 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [8]:
torch.save(f = "e401k_cocycles_results_splits={0}_boostrap={1}_trials={2}.pt".format(splits,bootstrap,len(seeds)), obj = results)